In [66]:
import pandas as pd
import numpy as np
import torch 
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
import prelim_process_df
from datetime import timedelta, datetime
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [11]:
df_abes_cls = pd.read_csv('../../../data/data_for_models/lstm_30m.csv',parse_dates = True,index_col = 'datetime')

In [12]:
df_abes_cls.head()

,close_avax,volume_avax,vol_usdt_K_avax,close_eth,volume_eth,vol_usdt_K_eth,close_btc,volume_btc,vol_usdt_K_btc,close_sol,volume_sol,vol_usdt_K_sol
datetime,,,,,,,,,,,,
2020-09-22 15:30:00,4.8811,2808316.25,13498.099902,342.05,20140.99740,6886.245163,10423.27,1305.659365,13609.113154,2.8000,61420.44,172.465165
2020-09-22 16:00:00,4.9312,1914636.29,9136.249619,339.99,23133.33317,7898.110882,10412.11,1147.850269,11969.341525,2.7540,53219.28,148.122713
2020-09-22 16:30:00,4.9096,1256377.31,6213.493286,336.12,48515.72194,16362.884815,10372.90,1801.381449,18703.921799,2.6728,158269.17,425.511003
2020-09-22 17:00:00,5.3951,1100528.85,5757.581164,339.58,32510.29604,11019.057797,10429.77,1388.738454,14468.525827,2.8134,267098.29,742.661738
2020-09-22 17:30:00,6.8219,2783452.58,17595.535036,342.11,23494.62347,8002.336092,10463.18,1288.599265,13468.384675,2.7602,95408.49,263.655611


In [13]:
def get_labels(df,c,days): 
    
    l = df.index
    final_id = l[-1]
    
    labels = {}
   
    for x in l:
        tmp1 = x + timedelta(days = days)
        if tmp1 < final_id:
            if (df.loc[tmp1][c] - df.loc[x][c])/(df.loc[x][c] + 0.00001) > 0.05:
                labels[x] = 1
            else:
                labels[x] = 0
        else:
            labels[x] = -1
    return(labels)        

In [14]:
df_abes_cls_labels = df_abes_cls.copy()

In [16]:
for c in tqdm(df_abes_cls_labels.columns):
    if c[:3] == 'clo':
        for i in [1,2,3]:
            a_tmp = pd.Series(get_labels(df_abes_cls_labels,c,i))
            c_tmp = c.split('close_')[1] + '_label_' + str(i)
            df_abes_cls_labels[c_tmp] = a_tmp 

  0%|          | 0/12 [00:00<?, ?it/s]

In [45]:
t = df_abes_cls.iloc[0:60,:].values.astype(float)

In [46]:
t.shape

(60, 12)

In [47]:
m = nn.Linear(12,16)

In [48]:
t1 = (torch.tensor(t))

In [49]:
x = m(t1.float())

In [52]:
x.size()

torch.Size([60, 16])

In [50]:
x.view(-1,1,16).size()

torch.Size([60, 1, 16])

In [ ]:
"""
df - dataframe with input variables,
labels - dataframe with output labels


"""

class lstm_dataset(Dataset):
    
    def __init__(self,df,labels):
        self.X = df.values
        self.labels = labels.values
        
    def __len__(self):
        return(len(self.X))
        
    def __getitem__(self,idx):    
        return(torch.tensor(self.X[idx]),torch.tensor(self.labels[idx]))

In [ ]:
train_dataloader = DataLoader(dset, batch_size=64, shuffle= False)

In [120]:
class LSTMpricepred(nn.Module):

    def __init__(self,input_dim, embedding_dim, hidden_dim, look_back):
        super(LSTMpricepred, self).__init__()
        self.hidden_dim = hidden_dim
        self.look_back = look_back
        self.linear_layer = nn.Linear(input_dim, embedding_dim)

        # The LSTM takes linear representations of each element of the input sequence, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2out = nn.Linear(hidden_dim,4)
        self.sigmoid = nn.Sigmoid()

    def forward(self, seq):
        embeds = self.linear_layer(seq)
        lstm_out = self.lstm(embeds.view(self.look_back, 1, -1))
        rep_space = self.hidden2out(lstm_out[1][0].view(hidden_dim))
        scores = self.sigmoid(rep_space)
        return scores

In [121]:
input_dim = 12
embedding_dim = 16
hidden_dim = 32
look_back = 60



model = LSTMpricepred(input_dim, embedding_dim, hidden_dim, look_back)

In [122]:
model(t1.float())

tensor([0.4167, 0.6128, 0.4551, 0.4443], grad_fn=<SigmoidBackward0>)

In [87]:
ll = nn.Linear(input_dim,embedding_dim)
lstm = nn.LSTM(embedding_dim,hidden_dim)

In [88]:
t2 = ll(t1)

In [89]:
t2.shape

torch.Size([60, 16])

In [90]:
t22 = t2.view(60,1,-1)

In [92]:
t3 = lstm(t22)

In [93]:
t3[0].size()

torch.Size([60, 1, 32])

In [97]:
t3[0][-1]

tensor([[ 0.7616,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.7616,  0.0000,
          0.0000,  0.0000,  1.0000,  1.0000,  0.0000,  0.0000, -0.7616, -1.0000,
         -0.0000, -0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         -0.7616, -0.0000,  0.0000,  0.0000,  0.0000, -0.0000,  0.0000,  1.0000]],
       grad_fn=<SelectBackward0>)

In [102]:
t3[1][0].view(32).size()

torch.Size([32])

In [103]:
ll2 = nn.Linear(32,4)

In [105]:
t4 = ll2(t3[1][0].view(32))

In [111]:
s = nn.Sigmoid()

In [112]:
s(t4)

tensor([0.5540, 0.4644, 0.5421, 0.4023], grad_fn=<SigmoidBackward0>)

In [69]:
t1 = torch.tensor(t).float()

In [71]:
model(t1).shape

torch.Size([60, 4])